## Final Exam BUS 458
Taiyu Zhang (200429688) ShuiTian Lian (200248618)

My partner and I collaborated on this task. For each question, we discussed different approaches together and then decided on what we believed was the most appropriate solution.

# Final Exam BUS 458 — Loan Data Analysis
**Objective:** Your objective is to identify the key drivers of loan approval and recommend which lender should be prioritized for each customer to maximize total payout

### **Instructions**
- Follow the prompts in each section of this notebook.
- **Where you see a “Question,” answer it directly below in a Markdown (text) cell — not as a code comment.**
- Your **reasoning, interpretations, and insights** should be written in text cells, clearly separated from code.
- You are encouraged to add extra **code cells**, **visualizations**, or **short explanations** if they strengthen your analysis or help you justify decisions.
- Make sure your notebook runs cleanly from start to finish without errors.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

Load Data (via Google Drive or direct upload)

In [3]:
# Add your code here
import pandas as pd
df = pd.read_csv("/content/sample_data/loan_data_analysis_final.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/loan_data_analysis_final.csv'

In [ ]:
# Check first 5 rows of dataframe
df.head(5)

In [ ]:
# Drop columns that have no variation or are unique

cols_to_drop = []

if "User ID" in df.columns:
    cols_to_drop.append("User ID")

for col in df.columns:
    if df[col].nunique() == 1:
        cols_to_drop.append(col)

df = df.drop(columns=cols_to_drop)

cols_to_drop

Understand the Data: Get a quick overview of the dataset structure and variable meanings before preprocessing.

In [ ]:
# Add your code here
df.info()
df.describe()
df.isna().sum()

## 1. Preprocessing


### 1.1 Handling Missing Values

**Question:** Why is median imputation often preferable to mean imputation for income and FICO?


**Answer:** Median imputation is not significantly affected by extreme cases such as minimum and maximum values. Both income and FICO data exhibit extreme cases and have many missing values.

In [ ]:
# Inspect missing values
df.isna().sum()

In [ ]:
# Apply imputation as instructed in the instructions pdf (create copies so original rows can be restored if needed)
# Impute

df_imputed = df.copy()

if 'FICO_score' in df_imputed.columns:
    df_imputed['FICO_score']= df_imputed['FICO_score'].fillna(df_imputed['FICO_score'].median())

if 'Monthly_Gross_Income' in df_imputed.columns:
    df_imputed['Monthly_Gross_Income']= df_imputed['Monthly_Gross_Income'].fillna(df_imputed['Monthly_Gross_Income'].median())

if 'Employment_Sector' in df_imputed.columns:
    df_imputed['Employment_Sector']= df_imputed['Employment_Sector'].fillna('Unknow')
df_imputed.isna().sum()

### 1.2 Outliers Detection and Removal

**Question:**
Which features had the most outliers, and what impact could they have on the model?

The Grant_Loan_Amount and Requested_Loan_Amnount had the most outlier.

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

numeric_cols = ['Granted_Loan_Amount', 'Requested_Loan_Amount', 'FICO_score', 'Monthly_Gross_Income', 'Monthly_Housing_Payment']

# Boxplots to highlight outliers for numerical columns

import matplotlib.pyplot as plt

numeric_cols = ['Granted_Loan_Amount', 'Requested_Loan_Amount', 'FICO_score', 'Monthly_Gross_Income', 'Monthly_Housing_Payment']

plt.figure(figsize=(8,4))
df_imputed [numeric_cols].boxplot()
plt.ylabel("Value")
plt.title("Boxplots of numeric features")

plt.xticks(rotation=45)


In [ ]:
# Remove Outliers using Z-score

from scipy import stats
import numpy as np

# Compute Z-scores for only numeric columns
z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))

# Choose threshold
threshold = 3  # common choice: 3 standard deviations

# Identify rows to keep (all Z-scores <= threshold)
rows_to_keep = (z_scores < threshold).all(axis=1)

# Track counts before/after
before_count = df.shape[0]
df_clean = df[rows_to_keep].copy()
after_count = df_clean.shape[0]

print(f"Outlier removal complete:")
print(f"Rows before: {before_count}")
print(f"Rows after:  {after_count}")
print(f"Rows removed: {before_count - after_count}")



In [ ]:
# BoxPlots after outlier removal
import matplotlib.pyplot as plt

numeric_cols = ['Granted_Loan_Amount', 'Requested_Loan_Amount', 'FICO_score', 'Monthly_Gross_Income', 'Monthly_Housing_Payment']

plt.figure(figsize=(8,4))
df_clean [numeric_cols].boxplot()
plt.ylabel("Value")
plt.title("Boxplots of numeric features after outlier removal")

plt.xticks(rotation=45)

**Question:**
Why do we need to remove or treat outliers? Provide reasoning based on model performance or data integrity.

**Answer** : The data from outliers is not representative and will affect model training.


**Question:**
Is there a difference between boxplots before and after outlier removal?

**Answer** : Removing outliers makes the scale smlng in clearer reading of the box plt and a more intuitive understanding of the data.

### 1.3 Redundancy & Correlation Check (5 Marks)

Detect duplicate data and multicollinearity.

**Question:**
Which numerical variables appear most strongly correlated with each other?

**Answer:** Requested Loan Amount and Granted Loan Amount have the strongest correlation.



**Question:**
Which numerical and categorical are strongly corelated to each other?

**Answer:** FICO Score and FICO SCORE Group are strongly corelated.


In [ ]:
# Correlation between numerical features
numeric_cols = ['FICO_score', 'Monthly_Gross_Income', 'Monthly_Housing_Payment', 'Granted_Loan_Amount','Requested_Loan_Amount', 'Approved']

# Plot correlation matrix, identify highly correlated pairs automatically (|corr|>0.95, excluding self-correlation)
corr = df_clean[numeric_cols].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt=".2f")
plt.title("Numeric Correlation Heatmap (with strongly correlated column)")
plt.show()


In [ ]:
# Correlation between Numeric and Categorical Variables (Correlation Ratio / η²)
from scipy import stats
import numpy as np
import pandas as pd

def correlation_ratio(categories, values):
    categories = pd.Series(categories)
    values = pd.Series(values)

    mask = categories.notna() & values.notna()
    categories = categories[mask].astype(str).values
    values = values[mask].astype(float).values

    cat_levels = np.unique(categories)
    overall_mean = np.mean(values)
    numerator = sum(len(values[categories == cat]) *
                    (np.mean(values[categories == cat]) - overall_mean) ** 2
                    for cat in cat_levels)
    denominator = sum((values - overall_mean) ** 2)
    return np.sqrt(numerator / denominator) if denominator != 0 else 0

# test all numeric–categorical pairs
num_cat_results = []


# Identify categorical and numerical columns
categorical_cols_clean = df_clean.select_dtypes(include='object').columns.tolist()
numeric_cols_clean = df_clean.select_dtypes(include=np.number).columns.tolist()


numeric_cols_for_eta = [col for col in numeric_cols_clean if col not in ['bounty', 'Approved']]
categorical_cols_for_eta = [col for col in categorical_cols_clean]


for num_col in numeric_cols_for_eta:
    for cat_col in categorical_cols_for_eta:
        eta = correlation_ratio(df_clean[cat_col], df_clean[num_col])
        num_cat_results.append((num_col, cat_col, eta))

num_cat_results = sorted(num_cat_results, key=lambda x: x[2], reverse=True)
print("Top 10 Numeric-Categorical Correlation Ratios (η²):")
for num_col, cat_col, eta in num_cat_results[:10]:
    print(f"{num_col} - {cat_col}: η² = {eta**2:.3f} (η = {eta:.3f})")


# Reorganize the correlation ratio results into a pivot table/matrix for heatmap
eta_matrix = pd.DataFrame(num_cat_results, columns=['Numerical_Feature', 'Categorical_Feature', 'Eta'])
eta_pivot = eta_matrix.pivot(index='Numerical_Feature', columns='Categorical_Feature', values='Eta')

# Plot the heatmap of correlation ratios (Eta)
plt.figure(figsize=(12, 8))
sns.heatmap(eta_pivot, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("Heatmap of Correlation Ratios (η) between Numeric and Categorical Features")
plt.xlabel("Categorical Features")
plt.ylabel("Numerical Features")
plt.tight_layout()
plt.show()

**Questions:** What features will you consider dropping after analyzing the graphs above?

**Answer** : I will delate Granted Loan Amount and FICO SCORE Group.



> *Note: For Decision Tree models, you do not need to remove or adjust for collinearity. The algorithm can naturally handle correlated features through its splitting mechanism. However, for Logistic Regression, multicollinearity can distort coefficient estimates and make interpretations unreliable. In that case, you may consider removing one variable from highly correlated pairs.


## 2. Exploratory Data Analysis

### 2.1 Target Variable Distribution

Analyze the distribution of the target variable (Approved) to understand class balance in loan approval outcomes.


**Question:**
Is the dataset balanced or imbalanced? Briefly explain the implication for modeling.


**Answer:**
This dataset is imbalanced. There are more cases with Approved = 0 than Approved = 1. It will tent predict Approved = 0.

In [ ]:
# Plot 'Approved' to visualise the count and balance
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3))
df_clean['Approved'].value_counts().plot(kind='bar')
plt.title("Distribution of Approved status")
plt.xlabel("Approved")
plt.ylabel("Count")


### 2.2 Distribution of Numerical Variables by Approval

Explore how key numerical variables differ between approved and rejected loan applications.


**Question:**
Which numerical variable is the most helpful variable ( most crucial predictor) ?

**Answer:**
According these graphs I will say the most helpful variale is FICO SCORE.


In [ ]:
# Visualise numerical variables against target variable

numerical_cols = ['Granted_Loan_Amount', 'FICO_score', 'Monthly_Gross_Income', 'Monthly_Housing_Payment']

for col in numerical_cols:
    plt.figure(figsize=(12,5))
    sns.histplot(
        data=df,
        x=col,
        hue='Approved',
        kde=True,
        stat='density',
        common_norm=False,
        palette='coolwarm',
        alpha=0.6
    )
    plt.title(f'Distribution of {col} by Approval Status')
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,4))
    sns.boxplot(
        data=df,
        x='Approved',
        y=col,
        hue='Approved',
        palette='coolwarm',
        legend=False
    )
    plt.title(f'Box Plot of {col} by Approval Status')
    plt.xlabel('Approved (0 = Denied, 1 = Approved)')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()


### 2.3 Categorical Variables and Approval

Analyze how categorical variables (such as Employment_Sector, Loan_Purpose, or Residence_Type) relate to the loan approval outcome.


**Question:**
Which categorical variable appears to have the strongest relationship with loan approval, and how can you tell?

**Answer:**
The strongest relationship to loan approval FICO score group. The highest approval rate is for the excellent level, the lowest approval rate is for poor group.

**Question:**
Are there any categories (e.g., Reason, Employment_Status) that appear to have minimal predictive value? Justify your answer with approval rate differences.

**Answer:**
Reason has extremely low predictive value. Regardlesss of the reason, their approval rates differ by the 10-12%, and this variation has little impact on the approval rate.  


In [ ]:
# Visualise categorical variables against target variable

categorical_cols = [
    'Reason', 'Fico_Score_group', 'Employment_Status',
    'Employment_Sector', 'Lender', 'Ever_Bankrupt_or_Foreclose'
]

for col in categorical_cols:
    approval_rates = df.groupby(col)['Approved'].mean().sort_values(ascending=False) * 100

    plt.figure(figsize=(10,5))
    sns.barplot(x=approval_rates.index, y=approval_rates.values, color='skyblue')
    plt.title(f'Approval Rate by {col}')
    plt.xlabel(col)
    plt.ylabel('Approval Rate (%)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"\nApproval Rate by {col}:\n{approval_rates.round(2)}")


### Cross Feature plots for multi-dimensional analysis.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# fico score and income
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df,x='FICO_score',y='Monthly_Gross_Income')
plt.title('FICO Score vs Monthly Gross Income by Approval')
plt.xlabel('FICO_score')
plt.ylabel('Monthly_Gross_Income')


# Monthly income vs Granted Loan Amount
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df,x='Monthly_Gross_Income',y='Granted_Loan_Amount')

plt.title('Monthly Gross Income vs Granted Loan Amount by Approval')
plt.xlabel('Monthly_Gross_Income')
plt.ylabel('Granted_Loan_Amount')


# Fico score vs Granted Loan Amount
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, x='FICO_score', y='Granted_Loan_Amount')
plt.title('FICO Score vs Granted Loan Amount by Lender')
plt.xlabel('FICO_score')
plt.ylabel('Granted_Loan_Amount')


## 3.5 Business Insights from EDA

After completing your visualizations and cross-feature analyses, summarize your findings in the context of loan approval and lender matching.  
Use this section to connect data patterns to real-world meaning before building predictive models.


**Question:** Which variables are the most helpful in understanding if a customer is going to be approved or denied for a loan?  

**Answer**: FICO score, Employemnet Status, Lender, Income, FICO core group.

**Question:** Are there any feature modifications or transformations that would improve the predictive power of a variable?  

**Answer**:

I will add the percentage of the monthly income spent on housing payment. It can improve the prefictive power. Different expendtures bring differrent risk for people with differnt income.

**Question:** What is each lender’s average approval rate?  

**Answer**:
Lender's each approval rate is
C    17.18%
A    10.91%
B     7.37%.

**Question:** Are there any clear differences between the three lenders in terms of which types of customers they approve?  

**Answer**: I just determine the differnece in approval rate between different lenders. However,I cannot make sure which types customers are approved.

**Question:** Are there variables that reliably predict a customer’s approval likelihood for a particular lender?  

**Answer**: I think income monthly, employment status, and monthly income percentage of specific expenditures are very powerful variables.




> Treat this section as your interpretation bridge between EDA and modeling. The goal is to show that you understand *why* certain patterns exist, not just that they exist.


## 3. Model Training

### 3.1 Train/Test Split and Encoding

In [ ]:
# Split dataset into train and test
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Create df_model from df_clean for preprocessing
df_model = df_clean.copy()

# --- Impute all remaining NaNs in df_model ---
# First, handle numerical columns
for col in df_model.select_dtypes(include=np.number).columns:
    if df_model[col].isnull().any():
        df_model[col] = df_model[col].fillna(df_model[col].median())

# Then, handle categorical/object columns
for col in df_model.select_dtypes(include='object').columns:
    if df_model[col].isnull().any():
        df_model[col] = df_model[col].fillna('Unknown_Category')

# Ensure 'Ever_Bankrupt_or_Foreclose' is treated as categorical for get_dummies if it has few unique values
if 'Ever_Bankrupt_or_Foreclose' in df_model.columns:
    df_model['Ever_Bankrupt_or_Foreclose'] = df_model['Ever_Bankrupt_or_Foreclose'].astype(str)

# Define target variable and features
y = df_model['Approved']
# Drop target variable and 'bounty' (if not used as feature)
X = df_model.drop(columns=['Approved', 'bounty'])

# Define categorical features to encode
# These columns should now be free of NaNs thanks to the imputation above
cat_feats = [
    'Reason',
    'Fico_Score_group',
    'Employment_Status',
    'Employment_Sector',
    'Lender',
    'Ever_Bankrupt_or_Foreclose' # Now string type
]

# One-hot encode categorical features
X = pd.get_dummies(X, columns=cat_feats, drop_first=True)

# Set RANDOM_STATE = 42 for reproducibility.
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

### 3.2 Logistic Regression Model
Build and interpret a Logistic Regression model to predict the likelihood of loan approval (Approved = 1).

**Question**
How accurate is the model on the test data, and what does this imply about its predictive strength?

**Answer**
The accurate was around 71.48%.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

logmodel = LogisticRegression(class_weight='balanced',random_state=42,solver='liblinear')
logmodel.fit(X_train, y_train)

In [ ]:
# Evaluate the model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
log_predictions = logmodel.predict(X_test)

test_accuracy = accuracy_score(y_test, log_predictions)
print("Test Accuracy:", test_accuracy )

print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, log_predictions))

print("\nLogistic Regression Confusion Matrix:")
print(confusion_matrix(y_test, log_predictions))

### 3.2 Decision Tree Classifier Model

Build and interpret a Decision Tree Classifier Model to predict the likelihood of loan approval (Approved = 1).



**Question**
How does the Decision Tree’s accuracy and AUC compare to the Logistic Regression model?

**Answer**
Add your answer here


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Train Decision Tree model

dtree = DecisionTreeClassifier(criterion="entropy",max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    max_features=None,
    class_weight='balanced',
    random_state=random_state)

dtree.fit(X_train, y_train)

In [ ]:
# Evaluate the model
tree_predictions = dtree.predict(X_test)
tree_proba = dtree.predict_proba(X_test)[:, 1]
tree_accuracy = accuracy_score(y_test, tree_predictions)
print("Decision Tree Test Accuracy:", tree_accuracy)

print("\nDecision Tree Classification Report:")
print(classification_report(y_test, tree_predictions))

print("\nDecision Tree Confusion Matrix:")
print(confusion_matrix(y_test, tree_predictions))

tree_auc = roc_auc_score(y_test, tree_proba)
print("\nDecision Tree ROC-AUC:", tree_auc)

In [ ]:
# Variable importance for Decision Tree
# Print top features
import pandas as pd
import seaborn as sns

feat_importances = pd.Series(dtree.feature_importances_, index=X_train.columns)
feat_importances = feat_importances.sort_values(ascending=False)
print("Top 10 Important Features (Decision Tree):")
print(feat_importances.head(10).round(4))

### Compare the performance of the Logistic Regression and Decision Tree models.


1.   Compare the performance of the Logistic Regression and Decision Tree
models.
2.   Evaluate and compare both models using key metrics such as ROC-AUC, Accuracy, Precision, Recall, and F1-score.
3. Plot and interpret the confusion matrices for both models. Discuss what false positives and false negatives mean in the business context (for example, approving risky applicants vs. rejecting qualified ones).
4. Identify which model best captures the target event (loan approval) and explain why.
5. Justify your model choice from both technical (performance, interpretability) and business (actionability, trust, and decision-making) perspectives.
6. Indicate the cutoff threshold you used and why it makes sense for this scenario.

In [ ]:
model = logmodel


In [ ]:
# Save your model as .pkl file for streamlit app development
import pickle

filename = 'my_model.pkl'  # Choose a path and descriptive filename with .pkl extension

# Open the file in binary write mode ('wb')
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved successfully to {filename}")


**Question:**
Which model do you recommend and why? (Provide both technical and business reasoning.)

**Answer:** I will use Logistic Regression Model, this model has the higest accuracy 71.48%. Logistic regression models can intuitively distinguish categorical variables, and in lending, they can more clearly show which variables affect the loan success rate, allowing lenders to gain a clearer understanding of their customers.

## 5. Recommendations
In this final section, summarize your key findings and provide concise business recommendations based on your analysis.


**Write a short paragraph including:**  
- A brief summary of your main findings from the analysis and model comparison.  
- The key factors that most influence loan approval.  
- Your recommendations on data or model based on the results  (for example, how approval decisions could be improved or better understood).  
- One next step you would take to improve the model or extend the analysis further.  


**Add your paragraph here:**
My analysis and model comparison show that both logistic regression and decision trees can predict loan approval results well, but logistic regression performs one percentage point better and is easier to interpret. Key factors influencing approval include credit quality FICO score/FICO score group, monthly income, housing payment, and whether there are any bankruptcy issues. Based on these results, I recommend using the logistic regression model as the primary decision support tool so that they can clearly understand which factors increase or decrease the probability of approval. Next, I can collect more data, add more detailed variables, and retrain the model to see if they can improve accuracy.



---



**Question:**  What trade-offs or risks should the company be aware of when using this model?  

**Answer:** The model's accuracy is not high enough, so we cannot rely entirely on it. Loan decisions are influenced by the economic environment, and these factors cannot be predicted by the model. Therefore, the company needs to consider external factors such as the economic environment when making decisions; the model can only provide a reference.

**Question:** How confident are you in your model’s generalizability — what might change its performance in real-world use?   

**Answer:**
I have confidence in the model, but I can't rely solely on it. Overfitting can affect model performance.

**Question:** If you had access to additional data, what new feature would you collect to strengthen this model?

**Answer:** I can collect the percentage of monthly income and expenses to enhance the model.

# Task
Generate multi-dimensional cross-feature plots using `seaborn.pairplot` to visualize relationships between 'FICO_score', 'Granted_Loan_Amount', 'Monthly_Gross_Income', and 'Monthly_Housing_Payment' with 'Approved' status as hue from the `df_clean` DataFrame. Additionally, create a grouped bar chart to explore the interaction between 'Fico_Score_group' and 'Lender' on loan approval rates using `df_clean`.

## generate_cross_feature_plots

### Subtask:
Generate multi-dimensional cross-feature plots using `seaborn.pairplot` to visualize relationships between key numerical features (FICO_score, Granted_Loan_Amount, Monthly_Gross_Income, Monthly_Housing_Payment) with 'Approved' status as hue. Additionally, create a grouped bar chart to explore the interaction between 'Fico_Score_group' and 'Lender' on loan approval rates.


## Summary:

### Data Analysis Key Findings
*   The `seaborn.pairplot` visualization was designed to reveal the distributions and pairwise correlations among `FICO_score`, `Granted_Loan_Amount`, `Monthly_Gross_Income`, and `Monthly_Housing_Payment`, differentiated by `Approved` status. This would highlight potential distinct patterns in these financial attributes for approved versus rejected loan applications.
*   The grouped bar chart was set up to illustrate loan approval rates across various `Fico_Score_group` categories, further segmented by `Lender`. This analysis would identify variations in lending behavior and approval rates among different lenders based on applicants' FICO scores.

### Insights or Next Steps
*   Analyzing these plots could provide insights into the key financial indicators and their thresholds that drive loan approval decisions, as well as how different lenders apply these criteria across various FICO score segments.
*   Further investigation could involve quantifying the identified relationships and discrepancies (e.g., specific FICO score cutoffs, income requirements, or lender-specific approval rates), potentially leading to a deeper understanding of loan eligibility criteria and lender policies.
